In [5]:
import librosa
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Đường dẫn tới thư mục chứa các tệp âm thanh của các thể loại nhạc khác nhau
audio_folder = './music'
# Danh sách các thể loại nhạc tương ứng với tên thư mục
genres = ['learning', 'podcast', 'pop','jazz','classical','rock', 'nolyric']

# Khởi tạo danh sách chứa các đặc trưng của âm thanh và nhãn thể loại
features = []
labels = []

# Trích xuất đặc trưng MFCC từ các tệp âm thanh và gán nhãn
for genre in genres:
    genre_folder = f'{audio_folder}/{genre}'
    for filename in os.listdir(genre_folder):
        if filename.endswith('.mp3'):
            file_path = f'{genre_folder}/{filename}'
            y, sr = librosa.load(file_path)
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            features.append(np.mean(mfccs, axis=1))  # Sử dụng trung bình của MFCCs làm đặc trưng
            labels.append(genre)

# Chuyển đổi nhãn về dạng số
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Huấn luyện một mô hình SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Dự báo nhãn thể loại cho tệp âm thanh kiểm tra
y_pred = svm_model.predict(X_test)

# Chuyển đổi dự báo về dạng nhãn ban đầu
predicted_labels = le.inverse_transform(y_pred)

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# In ra dự báo thể loại cho từng tệp âm thanh kiểm tra
for i, filename in enumerate(os.listdir(audio_folder)):
    if filename.endswith('.wav'):
        print(f'File: {filename}, Predicted Genre: {predicted_labels[i]}')


FileNotFoundError: [WinError 3] The system cannot find the path specified: './music/classical'